*****************************************************************
#  The Social Web: data representation
- Instructors: Jacco van Ossenbruggen, Dayana Spagnuelo
- TAs Michael Accetto, Oktay Kavi, Abhirup Mukherjee, Nihat Uzunalioğlu
- Exercises for Hands-on session 2
*****************************************************************

In this session you are going to mine data in various microformats. You will see the differences in what each of the formats can contain and what purpose they serve. We will start by looking at geographical data.

Prerequisites:
- Python 3.8
- Python packages: requests, BeautifulSoup4, HTMLParser, rdflib


In [1]:
# If you're using a virtualenv, make sure it's activated before running
# this cell!
!pip3 install requests
!pip3 install BeautifulSoup4
!pip3 install HTMLParser
!pip3 install rdflib

##  Exercise 1

Even if web pages do not use microformat, interesting data can often be extracted from the HML.You may use packages such as BeautifulSoup to extract arbitrary pieces of data from any HTML page.
The example below shows how we can find the URL of first image in the infobox table of the wikipedia page on Amsterdam. Tip: compare the code below with HTML source code of the wikipedia page: the image url is in the "src" attribute of the "img" element of in the "table" element with class="infobox".

In [2]:
# -*- coding: utf-8 -*-

import requests
from bs4 import BeautifulSoup

# This script requires you to add a url of a page with geotags to the commandline, e.g.
# python geo.py 'http://en.wikipedia.org/wiki/Amsterdam'
URL = 'https://en.wikipedia.org/wiki/Amsterdam'

req = requests.get(URL, headers={'User-Agent' : "Social Web Course Student"})
soup = BeautifulSoup(req.text)
# print(req.text)
image1 = soup.findAll('table', class_='infobox')[0].find('img')
print(image1['src'])  


//upload.wikimedia.org/wikipedia/commons/thumb/b/be/KeizersgrachtReguliersgrachtAmsterdam.jpg/270px-KeizersgrachtReguliersgrachtAmsterdam.jpg


Extracting coordinates from a webpage and reformatting them in the geo microformat (based on Example 8-1 in Mining the Social Web). Note that wikipages may encode long/lat information in different ways. On of the ways used by the Amsterdam wikipedia page is in a span element that is not shown to the user: 
<span class="geo">52.367; 4.900</span>
This span element has a single child: len(geoTag == 1) and no further structure, we have to manually get the long/lat by splitting the string on the ';' semicolon.

In [3]:

geoTag = soup.find(True, 'geo')
print(geoTag)

if geoTag and len(geoTag) > 1:
        lat = geoTag.find(True, 'latitude').string
        lon = geoTag.find(True, 'longitude').string
        print ('Location is at'), lat, lon
elif geoTag and len(geoTag) == 1:
        (lat, lon) = geoTag.string.split(';')
        (lat, lon) = (lat.strip(), lon.strip())
        print (('Location is at'), lat, lon)
else:
        print ('Location not found')


<span class="geo">52.367; 4.900</span>
Location is at 52.367 4.900


### Task 1

Can you convert the output of Exercise 1 into KML? Here is the KML documentation: https://developers.google.com/kml/documentation/?csw=1 and here you can find a simple example of how it is used: https://renenyffenegger.ch/notes/tools/Google-Earth/kml/index

Visualise the point in Google Maps using the following code example: https://developers.google.com/maps/documentation/javascript/examples/layer-kml-features
You will have to create your own KML file for the custom map layer, and provide a URL to the KML file inside the JavaScript code, which means that you have to upload the file somewhere. You can use a service like http://pastebin.com/ to obtain a URL for your KML file —> paste the code there and request the RAW format URL; use this one in this Task1.

Is KML a microformat, why (not)?

KML is a web-based data format and it's only purpose is to provide metadata. Such as the description of a placemark, the coordinates of a placemark, etc.

So KML is a microformat. 

### Pastebin link

https://pastebin.com/raw/T8mTYMdV

<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Placemark>
    <name>Simple placemark</name>
    <description>Attached to the ground. Intelligently places itself 
       at the height of the underlying terrain.</description>
    <Point>
      <coordinates>4.895168,52.370216</coordinates>
    </Point>
  </Placemark>
</kml>

### Jsfiddle link

https://jsfiddle.net/m1pg736j/

## Exercise 2 
In order to find information in the web we can use microformats. However in this example you will not be using hRecipe. Instead, we'll show you how to find arbitrary tags in a webpage.


### Task 2 
Parsing data for a <sub><sup>veggie</sup></sub> spaghetti alla carbonara recipe (from Example 2-7 in Mining the Social Web).

In [3]:
import requests
import json
from bs4 import BeautifulSoup

# A yummy webpage (feel free to change to your likings.)
URL = "https://www.jamieoliver.com/recipes/pasta-recipes/veggie-carbonara/"

# requests will return the html found at the given webpage...
page = requests.get(URL)
# ...and a BeautifulSoup object can be created from its content.
soup = BeautifulSoup(page.content, 'html.parser')

listchildren = list(soup.children)
print(listchildren)

['html', '\n', <html class="single-recipe country-code-default" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link crossorigin="" href="https://cdn.jamieoliver.com" rel="preconnect"/>
<link crossorigin="" href="//img.jamieoliver.com" rel="preconnect"/>
<link crossorigin="" href="//fonts.googleapis.com" rel="preconnect"/>
<link crossorigin="" href="//code.jquery.com" rel="preconnect"/>
<!-- Meta -->
<title>Vegetarian carbonara recipe | Jamie Oliver pasta recipes</title>
<meta content="Think that carbonara needs bacon? Think again! This vegetarian carbonara recipe uses leafy greens to make a beautiful pea and spinach carbonara. Happy days!" name="description">
<meta content="Jamie Oliver, recipes, food, pasta, lasagne, videos" name="keywords">
<meta content="JamieOliver.com" name="author">
<meta content="050179ea6bc8e9e62d0fd9edcc154d4a" name="p:domain_verify"/>
<meta

We can find any element in the page through *css tag selectors*
You can find them all [here](https://www.w3schools.com/cssref/css_selectors.asp), but shortly these are "." for classes, # for ids and plain text for the element name.


You can also combine them, so that looking for ".class1.class2" would select all elements displaying both classes. For a deeper overview please check the above link (or google "html tag selectors"). 

In [6]:
print(len(listchildren)) # we can see here how many children the html doc has got.
ingredients_unparsed = soup.select_one(".tasty-recipes-ingredients")
# let's get all the "list item" elements in a list:
ing_unp = ingredients_unparsed.findAll('li')
print(ing_unp)

4
[<li><span data-amount="1">1</span> pound spaghetti noodles</li>, <li><span data-amount="0.5" data-unit="cup">½ cup</span> smoked mozzarella cheese</li>, <li><span data-amount="0.5" data-unit="cup">½ cup</span> grated Parmesan cheese, plus more for serving</li>, <li><span data-amount="4">4</span> egg yolks</li>, <li><span data-amount="1" data-unit="cup">1 cup</span> frozen Earthbound Farm Organic peas</li>, <li><span data-amount="8" data-unit="cup">8 cups</span> Earthbound Farm Organic spinach</li>, <li><span data-amount="3" data-unit="tablespoon">3 tablespoons</span> butter</li>, <li><a class="tasty-link" data-tasty-links-no-disclosure="" href="https://www.acouplecooks.com/what-is-kosher-salt/" target="_blank">Kosher salt</a></li>, <li>Fresh ground black pepper</li>]


Mmmh... not so pretty yet. How about listing their items using the text method?

In [17]:

ingredients = [t.text for t in ing_unp]
print("Ingredients:\n")
# [print(i) for i in ingredients]  # Also prints the generator
# Instead
for ing in ingredients:
    print(ing)

Ingredients:

1 pound spaghetti noodles
½ cup smoked mozzarella cheese
½ cup grated Parmesan cheese, plus more for serving
4 egg yolks
1 cup frozen Earthbound Farm Organic peas
8 cups Earthbound Farm Organic spinach
3 tablespoons butter
Kosher salt
Fresh ground black pepper


Good. Now the instructions:

In [8]:
instructions_unparsed = soup.select_one(".tasty-recipes-instructions")
instructions_unparsed = instructions_unparsed.findAll("li")
print(instructions_unparsed)

[<li id="instruction-step-1">In a large pot, combine 6 quarts of water with 2 tablespoons <a class="tasty-link" data-tasty-links-no-disclosure="" href="https://www.acouplecooks.com/what-is-kosher-salt/" target="_blank">kosher salt</a> and bring it to a boil.</li>, <li id="instruction-step-2">Grate the Parmesan and mozzarella cheese. Carefully separate four egg yolks and set aside.</li>, <li id="instruction-step-3">Once boiling, add the pasta and cook until the pasta is just about al dente, about 7 minutes; then add peas and spinach and cook for 1 minute. Reserve 1 cup cooking water, and then drain the pasta and vegetables.</li>, <li id="instruction-step-4">In a skillet, melt the butter, then stir in the cheeses, ¼ cup pasta water, and ¼ teaspoon <a class="tasty-link" data-tasty-links-no-disclosure="" href="https://www.acouplecooks.com/what-is-kosher-salt/" target="_blank">kosher salt</a>. Stir in the pasta and vegetables until creamy over low heat, adding more pasta water if necessary 

Let's finish off with the title:

In [9]:
title_unparsed = soup.select_one(".post-header") # 
categorical_title = title_unparsed.text.split("›") # website specific divider.
recipe_title = categorical_title[-1].strip() # let's remove that ugly space at the beginning.
recipe_title

'Vegetarian Carbonara'

## Task 2.1
Now it's your turn. Create a function that can scrape any recipe webpage from the same website (other websites will have different class tags). 

Make sure to:

- return itemized content (e.g. ingredients) in a list. You may want to use a list comprehension here.
- Not all items have been cleaned of their html markdown (see variables ```ingredients``` vs. ```instructions_unparsed```. Make sure to return a list with human readable content (i.e. by using the ```.text``` attribute).


In [37]:
# -*- coding: utf-8 -*-

import requests
import json
from bs4 import BeautifulSoup

# Pass in a URL containing hRecipe, such as
# https://www.jamieoliver.com/recipes/pasta-recipes/veggie-carbonara/

URL = "https://www.jamieoliver.com/recipes/mushroom-recipes/mushroom-stroganoff/"#YOUR RECIPE HERE/

# Parse out some of the pertinent information for a recipe.
# See http://microformats.org/wiki/hrecipe.

def parse_website(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # You code here
    # Parse header and get the title
    title_unparsed = soup.select_one(".single-recipe-title") # 
    categorical_title = title_unparsed.text.split(",") # website specific divider.
    recipe_title = categorical_title[-1].strip() # let's remove that ugly space at the beginning.
    fn = recipe_title

    # Ingredients
    ingredients_unparsed = soup.select_one(".recipe-ingredients")
    # let's get all the "list item" elements in a list:
    ing_unp = ingredients_unparsed.findAll('li')
    ingredients = [t.text for t in ing_unp]
    list_ingredients = []
    for ing in ingredients:
        list_ingredients.append(ing)

    # Instructions
    instructions_unparsed = soup.select_one(".recipe-instructions")
    instructions_splitted = instructions_unparsed.findAll("li")
    instructions = [t.text for t in instructions_splitted]

    return {
        'name': fn,
        'ingredients': list_ingredients, 
        'instructions': instructions,
    }

recipe = parse_website(URL)
print (recipe)
#lol groetjes sebas

{'name': 'Mushroom stroganoff', 'ingredients': ['\n                                                                                                            400                                                                                                                                                                g                                                                                                                                                                mixed mushrooms                                                                                                                                                        ', '\n                                                                                                            1                                                                                                                                                                                                                    red onion              

But How can we get information not only from one website,  but from all? 

The answer: microformats.

But rather than extracting with information manually from the schema.org or hRecipe microformats, we can use a package, ```scrape-schema-recipe``` 

Feel free to experiment with it. 

### Task 2.2
hRecipe is a microformat specifically created for recipes.
Can you for example easily compare different dessert recipe ingredients? For inspiration you can look back at the exercises you did in Hands-on session 1 where you compared different sets of tweets.

### Dessert 1

In [38]:
URL_dessert_1 = "https://www.jamieoliver.com/recipes/cake-recipes/drizzle-bundt-cake/"

recipe = parse_website(URL_dessert_1)
print (recipe)

{'name': 'Drizzle bundt cake', 'ingredients': ['\n                                                                                                            375                                                                                                                                                                g                                                                                                                                                                unsalted butter (at room temperature)                                                                                                                                                                , plus extra for greasing                                                                                                     ', '\n                                                                                                            375                                                                              

### Dessert 2

In [39]:
URL_dessert_2 = "https://www.jamieoliver.com/recipes/fruit-recipes/homemade-candied-peel/"

recipe = parse_website(URL_dessert_2)
print (recipe)

{'name': 'Homemade candied peel', 'ingredients': ['\n                                                                                                                                                                                                                    rinds of 2 lemons                                                                                                                                                        ', '\n                                                                                                                                                                                                                    rinds of 2 oranges                                                                                                                                                        ', '\n                                                                                                                                                                            

## Exercise 3

Schema.org is one of the most widely used annotations formats. Schema.org is a multipurpose  template that has been created by a consortium consisting of Yahoo!, Google and Microsoft. It can describe entities, events, products etc. Check out the vocabulary specs on Schema.org.

### Task 3

Parsing schema.org microdata. To parse this data you need to install the rdflib-microdata package, which you have done in one of the previous steps.



In [ ]:
from rdflib import Graph

# Source: https://www.youtube.com/watch?v=sCU214rbRZ0
# Pass in a URL containing Schema.org microformats
URL = "http://dbpedia.org/resource/Micheal_Jackson"

# Initialize a graph
g = Graph()

# Parse in an RDF file graph dbpedia
result = g.parse(location=URL)

# Loop through first 10 triples in the graph
for index, (sub, pred, obj) in enumerate(g):
    print(sub, pred, obj)
    if index == 10:
        break

In [ ]:
# Print the size of the Graph
print(f'Graph has {len(g)} facts')

In [ ]:
# Print out the entire Graph in the RDF Turtle format
print(g.serialize(format='ttl').decode('u8'))

### Task 3.1 
Compare the schema.org information about a band on last.fm to the Facebook Open Graph information about the same band from Facebook. What are the differences? Which format do you think supports better interoperability?

### Last.fm

In [42]:
URL = "https://www.last.fm/music/Coldplay"

# requests will return the html found at the given webpage...

# ...and a BeautifulSoup object can be created from its content.
#searched_word = 'class="wiki"'

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
result_1 = soup.find_all("div", itemtype="http://schema.org/MusicGroup")
result_2 = soup.find_all("div", itemtype="http://schema.org/MusicRecording")

print(result_1)
print(result_2)

[<div data-page-resource-blacklist-level="" data-page-resource-name="Coldplay" data-page-resource-type="artist" itemscope="" itemtype="http://schema.org/MusicGroup">
<div data-require="components/disclose-base,components/disclose-autoclose-v2,components/disclose-dropdown-v2,components/disclose-dropdown-location-picker-v2,components/disclose-collapsing-nav-v2,components/disclose-artwork,components/disclose-remove,components/disclose-search,components/disclose-hover-v3,components/disclose-select,components/disclose-lazy-buylinks,components/focus-controls,components/prevent-resubmit-v2,components/edit-scrobble,components/toggle-buttons,components/click-proxy,components/bookmark-notification"></div>
<nav class="masthead"><div class="masthead-inner-wrap"><div class="masthead-logo"><span class="masthead-logo-loading"></span><a href="/"> Last.fm</a></div><a aria-controls="masthead-search" class="masthead-search-toggle" data-disclose-search="" href="/search">Search</a><form action="/search" cl

### Facebook

In [43]:
URL = "https://www.facebook.com/15253175252"

# requests will return the html found at the given webpage...

# ...and a BeautifulSoup object can be created from its content.
#searched_word = 'class="wiki"'

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
result_1 = soup.find_all("div", itemtype="http://schema.org/MusicGroup")
result_2 = soup.find_all("div", itemtype="http://schema.org/MusicRecording")

print(result_1)
print(result_2)

[]
[]


### Task 3.2
Explore the various microformats at http://microformats.org/ and compare the output of the exercises with the output of http://microformats.org/. Think about possible microformats you want to support in your final assignment and read up on how to parse them.